In [1]:
import gradio as gr
import pickle
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from meteostat import Point, Daily
import geohash2
from utils_main_holidays import extract_public_Holiday_v2
import os
import tempfile
import platform
import joblib

# Load your trained model and encoder
try:
    # with open('best_model_XGBoost.pkl', 'rb') as f:
    #     model = pickle.load(f)
    
    # with open('onehot_encoder.pkl', 'rb') as f:
    #     encoder = pickle.load(f)
    
    # Load network and location options from CSV
    network_df = pd.read_csv('network_list.csv')
    location_df = pd.read_csv('location_list.csv')
    otcRatio = pd.read_csv('jeki_truth_data_reference.csv')
    holiday = pd.concat([extract_public_Holiday_v2(year, "JP") for year in [2024, 2025]])
    
    print("All files loaded successfully!")
    
except Exception as e:
    print(f"Error loading files: {e}")
    # Create dummy data for testing if files don't exist
    network_df = pd.DataFrame({'NetworkId': ['Network1', 'Network2'], 'ReferenceId': ['Ref1', 'Ref2']})
    location_df = pd.DataFrame({
        'ReferenceId': ['Ref1', 'Ref2'], 
        'lat': [35.6762, 35.6895], 
        'lon': [139.6503, 139.6917],
        'spotsPerHour': [4, 6],
        'spotDuration': [15, 20],
        'dwellTime': [30, 45],
        'loopLength': [120, 180]
    })
    otcRatio = pd.DataFrame({
        'referenceId': ['Ref1', 'Ref2'],
        'share': [0.8, 0.9],
        'mediaRatio': [0.7, 0.8]
    })
    holiday = pd.DataFrame({'date': [], 'Day_Type_2': []})

# Generate geohash5 and geohash6 columns from lat/lon
location_df['geohash5'] = location_df.apply(lambda row: geohash2.encode(row['lat'], row['lon'], precision=5), axis=1)
location_df['geohash6'] = location_df.apply(lambda row: geohash2.encode(row['lat'], row['lon'], precision=6), axis=1)

network_names = sorted(network_df['NetworkId'].unique().tolist())
location_names = sorted(location_df['ReferenceId'].unique().tolist())

# Spot calculation functions
def check_spot(spotsHour, spotDur):
    return spotsHour * (spotDur / (spotDur + 15))

def spotPV(spotDurMult, spotGet, avgPerMin, pv):
    return min(spotDurMult * spotGet * avgPerMin, pv)

def check_dwell(dwellTime, loopLength):
    if dwellTime < 10:
        return 0.6
    elif dwellTime >= 10 and dwellTime >= loopLength:
        return dwellTime / loopLength
    elif dwellTime >= 10 and dwellTime < loopLength:
        return 1

def spot_calc(pv, dwellTime, loopLength, spotsHour, spotDur):
    avgPerMin = pv / 60 if dwellTime <= 60 else (pv / 60) * (dwellTime / 60)
    spotGet = check_dwell(dwellTime, loopLength)
    spotDurMult = check_spot(spotsHour, spotDur)
    return spotPV(spotDurMult, spotGet, avgPerMin, pv)

def custom_round(value):
    return round(value, 2) if pd.notnull(value) else 0

# Global cart to hold campaign items
campaign_cart = []

# Store the latest report data for downloads
latest_report_data = None

# Excel report generation functions (from utils_main_report.py)
def displayName(name):
    loc = {'JPN-JEK-D-00000-00029': 'J・ADビジョン　巣鴨駅改札外',
           'JPN-JEK-D-00000-00030': 'J・ADビジョン　新宿駅東口',
           'JPN-JEK-D-00000-00031': 'J・ADビジョン　新宿駅南口',
           'JPN-JEK-D-00000-00032': 'J・ADビジョン　新宿駅甲州街道改札',
           'JPN-JEK-D-00000-00034': 'J・ADビジョン　五反田駅',
           'JPN-JEK-D-00000-00035': 'J・ADビジョン　品川駅中央改札内',
           'JPN-JEK-D-00000-00039': 'J・ADビジョン　有楽町駅中央改札口',
           'JPN-JEK-D-00000-00040': 'J・ADビジョン　東京駅丸の内地下連絡通路',
           'JPN-JEK-D-00000-00041': 'J・ADビジョン　東京駅京葉通路',
           'JPN-JEK-D-00000-00042': 'J・ADビジョン　秋葉原駅新電気街口',
           'JPN-JEK-D-00000-00044': 'J・ADビジョン　吉祥寺駅南北自由通路',
           'JPN-JEK-D-00000-00045': 'J・ADビジョン　浦和駅改札口',
           'JPN-JEK-D-00000-00046': 'J・ADビジョン　大宮駅中央改札',
           'JPN-JEK-D-00000-00047': 'J・ADビジョン　横浜駅中央通路',
           'JPN-JEK-D-00000-00048': 'J・ADビジョン　JR横浜タワーアトリウム',
           'JPN-JEK-D-00000-00049': 'J・ADビジョン　高田馬場駅スマイル・ステーションビジョン',
           'JPN-JEK-D-00000-00050': 'J・ADビジョン　池袋駅中央改札内',
           'JPN-JEK-D-00000-00051': 'J・ADビジョン　桜木町駅',
           'JPN-JEK-D-00000-00052': 'J・ADビジョン　横浜駅南改札内',
           'JPN-JEK-D-00000-00058': 'J・ADビジョン　東京駅新幹線北乗換口',
           'JPN-JEK-D-00000-00059': 'J・ADビジョン　東京駅新幹線南乗換口',
           'JPN-JEK-D-00000-00060': 'J・ADビジョン　恵比寿駅西口',
           'JPN-JEK-D-00000-00061': 'J・ADビジョン　赤羽駅北改札',
           'JPN-JEK-D-00000-00960': 'J・ADビジョン　八王子駅自由通路南',
           'JPN-JEK-D-00000-00961': 'J・ADビジョン　上野駅公園改札内',
           'JPN-JEK-D-00000-04333': 'J・ADビジョン 新橋駅北改札',
           'JPN-JEK-D-00000-04334': 'J・ADビジョン 新橋駅南改札',
           'JPN-JEK-D-00000-00036': 'J・ADビジョン　高輪ゲートウェイ駅',
           'JPN-JEK-N-00000-00055': '[2025]J・adビジョン ステーションネットワーク'}
    return loc.get(name, name)

def networkSum_(df, name):
    df_copy = df.copy()
    df_copy[['gender', 'age']] = df_copy['agegender'].str.split('_', n=1, expand=True)
    df_copy['age'] = df_copy['age'] + '_Impressions'
    df_copy['gender_age'] = df_copy['gender'] + '_' + df_copy['age']
    
    # Group and sum PV
    network_df = df_copy.groupby(['date', 'hour_group', 'gender_age'])['PV'].sum().reset_index()
    
    # Pivot to wide format
    pivot_df = network_df.pivot_table(index=['date', 'hour_group'],
                                     columns='gender_age', values='PV', fill_value=0).reset_index()
    
    # Fill missing columns
    expected_cols = [f'{gender}_{age}_Impressions' for gender in ['female', 'male'] for age in ['10_19', '20_29', '30_39', '40_49', '50_59', '60_plus']]
    for col in expected_cols:
        if col not in pivot_df:
            pivot_df[col] = 0
    
    # Reorder columns
    location = displayName(name)
    pivot_df['Location'] = location
    pivot_df['Reference_Id'] = name
    pivot_df = pivot_df[['Location', 'date', 'hour_group', 'Reference_Id'] + expected_cols]
    
    # Calculate Total PV
    pivot_df['Total Impressions'] = pivot_df[expected_cols].sum(axis=1)
    impression_cols = expected_cols + ['Total Impressions']
    pivot_df[impression_cols] = pivot_df[impression_cols].astype(int)
    pivot_df = pivot_df.sort_values(by=['date', 'hour_group'])
    pivot_df['date'] = pd.to_datetime(pivot_df['date'])

    # Format date nicely
    if platform.system() == 'Windows':
        pivot_df['date'] = pivot_df['date'].dt.strftime('%A, %#d %B, %Y')
    else:
        pivot_df['date'] = pivot_df['date'].dt.strftime('%A, %-d %B, %Y')

    return pivot_df

def ageGender_script(df):
    # Age breakdown
    age_order = ['10-19', '20-29', '30-39', '40-49', '50-59', '60_plus']
    df_copy = df.copy()
    df_copy['age'] = df_copy['agegender'].str.extract(r'(\d+)', expand=False).astype(int)
    df_copy['age_group'] = pd.cut(df_copy['age'], bins=[9, 19, 29, 39, 49, 59, 200], 
                                 labels=age_order, right=True)
    
    age_summary = df_copy.groupby('age_group')['PV'].sum().reset_index()
    age_summary.columns = ['Age', 'Impression']
    age_summary['Percentage'] = (age_summary['Impression'] / age_summary['Impression'].sum())
    age_summary = age_summary[['Age', 'Percentage', 'Impression']]
    
    # Gender breakdown
    df_copy['gender'] = df_copy['agegender'].str.extract(r'(female|male)')
    gender_summary = df_copy.groupby('gender')['PV'].sum().reset_index()
    gender_summary.columns = ['Gender', 'Impression']
    gender_summary['Percentage'] = (gender_summary['Impression'] / gender_summary['Impression'].sum())
    gender_summary = gender_summary[['Gender', 'Percentage', 'Impression']]

    # Age-Gender breakdown
    agegender_summary = df_copy.groupby('agegender')['PV'].sum().reset_index()
    agegender_summary.columns = ['AgeGender', 'Impression']
    agegender_summary['Percentage'] = agegender_summary['Impression'] / agegender_summary['Impression'].sum()
    
    agegender_pivot = agegender_summary.set_index('AgeGender')[['Percentage', 'Impression']].T
    agegender_pivot.loc['Percentage'] = agegender_pivot.loc['Percentage'].astype(float).round(10)
    agegender_pivot.loc['Impression'] = agegender_pivot.loc['Impression'].apply(lambda x: f"{int(x):,}")
    
    # Age-Gender per location
    agegender_overall = df_copy.groupby(['Reference_Id', 'agegender'])['PV'].sum().reset_index()
    agegender_overall.columns = ['Reference_Id', 'AgeGender', 'Impression']
    agegender_overall['Percentage'] = agegender_overall.groupby('Reference_Id')['Impression'].transform(lambda x: x / x.sum())
    agegender_overall = agegender_overall.pivot(index='Reference_Id', columns='AgeGender', values=['Impression', 'Percentage'])
    
    return age_summary, gender_summary, agegender_pivot, agegender_overall

def generate_excel_report(df):
    """Generate Excel report using the utils_main_report.py logic"""
    try:
        # Prepare data
        df_report = df.copy()
        
        # Map hours to hour groups
        hour_groups_train_channel = {
            5: '05-10', 6: '05-10', 7: '05-10', 8: '05-10', 9: '05-10', 
            10: '10-18', 11: '10-18', 12: '10-18', 13: '10-18', 14: '10-18', 
            15: '10-18', 16: '10-18', 17: '10-18', 
            18: '18-24', 19: '18-24', 20: '18-24', 21: '18-24', 22: '18-24', 23: '18-24'
        }
        
        df_report["hour_group"] = df_report["hour"].map(hour_groups_train_channel)
        df_report = df_report.rename(columns={"SpotImpressions": "PV"})
        df_report['date'] = pd.to_datetime(df_report['date'])
        
        # Create temporary file
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx')
        
        with pd.ExcelWriter(temp_file.name, engine='openpyxl') as writer:
            # Sheet 1: Overall Performance
            overall_perf = df_report.groupby("ReferenceId", as_index=False)["PV"].sum()
            overall_perf = overall_perf.rename(columns={"PV": "Impression"})
            overall_perf.to_excel(writer, sheet_name="Overall Performance", index=False)
            
            # Sheet 2: Daily Summary
            daily = df_report.groupby(["date"], as_index=False)["PV"].sum()
            daily = daily.rename(columns={"PV": "Impression"})
            daily['date'] = daily['date'].dt.strftime('%Y-%m-%d')
            daily.to_excel(writer, sheet_name="Daily Summary", index=False)
            
            # Sheet 3: Age Gender Analysis
            df_report_renamed = df_report.rename(columns={"ReferenceId": "Reference_Id"})
            age_summary, gender_summary, agegender_pivot, agegender_overall = ageGender_script(df_report_renamed)
            age_summary.to_excel(writer, sheet_name="Age Gender", startrow=0, index=False)
            gender_summary.to_excel(writer, sheet_name="Age Gender", startrow=0, startcol=4, index=False)
            agegender_pivot.to_excel(writer, sheet_name="Age Gender", startrow=10)
            agegender_overall.to_excel(writer, sheet_name="Age Gender", startrow=15)

            # Sheet 4: Hourly Summary
            hourly = df_report.groupby(["hour"], as_index=False)["PV"].sum()
            hourly = hourly.rename(columns={"PV": "Impression"})
            hourly.to_excel(writer, sheet_name="Hourly Summary", index=False)

            # Sheet 5: Network Summary
            overall_name = 'JPN-JEK-N-00000-00055'
            network_summary = networkSum_(df_report_renamed, overall_name)
            network_summary.to_excel(writer, sheet_name="Network Summary", index=False)

            # Individual location sheets
            reference_ids = df_report['ReferenceId'].unique()
            for ref_id in reference_ids:
                tmp = df_report_renamed[df_report_renamed['Reference_Id'] == ref_id]
                if not tmp.empty:
                    network_summary = networkSum_(tmp, ref_id)
                    loc_name = displayName(ref_id)
                    # Sanitize sheet name for Excel (max 31 chars, remove invalid chars)
                    sheet_name = loc_name[:31].replace('/', '_').replace('\\', '_').replace('?', '_').replace('*', '_').replace('[', '_').replace(']', '_')
                    network_summary.to_excel(writer, sheet_name=sheet_name, index=False)
        
        return temp_file.name
        
    except Exception as e:
        print(f"Error generating Excel report: {e}")
        return None

def get_weather_label(date_str, lat, lon):
    try:
        date = datetime.strptime(date_str, "%Y-%m-%d")
        point = Point(lat, lon)
        daily = Daily(point, date, date)
        weather = daily.fetch()
        if weather.empty:
            return "sunny"  # default fallback
        w = weather.iloc[0]
        prcp = w.get('prcp', 0) or 0
        snow = w.get('snow', 0) or 0
        tavg = w.get('tavg', 15) or 15
        if snow > 0:
            return "snow"
        elif prcp > 0:
            return "rain"
        elif tavg < 5:
            return "cold"
        else:
            return "sunny"
    except Exception as e:
        print(f"Weather API error: {e}")
        return "sunny"  # fallback

def hour_type_to_hours(hour_type):
    mapping = {
        'morning': list(range(5, 12)),
        'afternoon': list(range(12, 18)),
        'evening': list(range(18, 24)),
        'night': list(range(0, 5)),
        'full': list(range(5, 24))
    }
    return mapping.get(hour_type.lower(), [])

def generate_date_range(start, end):
    return pd.date_range(start=start, end=end)

def add_to_cart(item_type, name, start_date, end_date, hour_types, spots_per_hour):
    if not name:
        return "Please select a name", get_cart_display(), update_remove_choices()
    
    if not start_date or not end_date:
        return "Please select both start and end dates", get_cart_display(), update_remove_choices()
    
    if not hour_types:
        return "Please select at least one hour type", get_cart_display(), update_remove_choices()
    
    try:
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.strptime(end_date, "%Y-%m-%d")
        if start_dt > end_dt:
            return "End date must be after start date", get_cart_display(), update_remove_choices()
    except ValueError:
        return "Invalid date format", get_cart_display(), update_remove_choices()
    
    item = {
        'type': item_type,
        'name': name,
        'start_date': start_date,
        'end_date': end_date,
        'hour_type': hour_types if isinstance(hour_types, list) else [hour_types],
        'spots_per_hour': int(spots_per_hour) if spots_per_hour else 1
    }
    
    campaign_cart.append(item)
    return f"Added: {item_type} - {name}", get_cart_display(), update_remove_choices()

def get_cart_display():
    if not campaign_cart:
        return pd.DataFrame(columns=['Type', 'Name', 'Start Date', 'End Date', 'Hour Types', 'Spots/Hour'])
    
    display_data = []
    for i, item in enumerate(campaign_cart):
        display_data.append({
            'Index': i,
            'Type': item['type'],
            'Name': item['name'],
            'Start Date': item['start_date'],
            'End Date': item['end_date'],
            'Hour Types': ', '.join(item['hour_type']),
            'Spots/Hour': item['spots_per_hour']
        })
    
    return pd.DataFrame(display_data)

def update_remove_choices():
    if not campaign_cart:
        return gr.update(choices=[])
    choices = [f"{i}: {item['type']} - {item['name']}" for i, item in enumerate(campaign_cart)]
    return gr.update(choices=choices)

def remove_from_cart(selected_item):
    if not selected_item or not campaign_cart:
        return "No item selected or cart is empty", get_cart_display(), update_remove_choices()
    
    try:
        index = int(selected_item.split(':')[0])
        if 0 <= index < len(campaign_cart):
            removed_item = campaign_cart.pop(index)
            return f"Removed: {removed_item['type']} - {removed_item['name']}", get_cart_display(), update_remove_choices()
        else:
            return "Invalid selection", get_cart_display(), update_remove_choices()
    except (ValueError, IndexError):
        return "Invalid selection format", get_cart_display(), update_remove_choices()

def clear_cart():
    campaign_cart.clear()
    return "Cart cleared successfully", get_cart_display(), update_remove_choices()

def predict(cart):
    if not cart:
        return pd.DataFrame()
    
    data = []
    agegender_keys = [
        'female_10_19', 'female_20_29', 'female_30_39', 'female_40_49', 'female_50_59', 'female_60_plus',
        'male_10_19', 'male_20_29', 'male_30_39', 'male_40_49', 'male_50_59', 'male_60_plus'
    ]

    for item in cart:
        try:
            if item['type'] == 'Location':
                location_data = location_df[location_df['ReferenceId'] == item['name']]
            elif item['type'] == 'Network':
                network_ref_ids = network_df[network_df['NetworkId'] == item['name']]['ReferenceId'].tolist()
                location_data = location_df[location_df['ReferenceId'].isin(network_ref_ids)]
            else:
                continue

            if location_data.empty:
                continue

            for _, loc in location_data.iterrows():
                date_range = generate_date_range(item['start_date'], item['end_date'])
                for date in date_range:
                    date_str = date.strftime('%Y-%m-%d')
                    weather_label = get_weather_label(date_str, loc['lat'], loc['lon'])
                    for hour_type in item['hour_type']:
                        for hour in hour_type_to_hours(hour_type):
                            for agegender in agegender_keys:
                                row = {
                                    'date': date_str,
                                    'Reference_Id': loc['ReferenceId'],
                                    'geohash5': loc['geohash5'],
                                    'geohash6': loc['geohash6'],
                                    'hour': hour,
                                    'agegender': agegender,
                                    'weather': weather_label,
                                    'spotsPerHour': item['spots_per_hour'],
                                    'spotDuration': loc['spotDuration'],
                                    'dwellTime': loc['dwellTime'],
                                    'loopLength': loc['loopLength']
                                }
                                data.append(row)
        except Exception as e:
            print(f"Error processing item {item}: {e}")
            continue

    if not data:
        return pd.DataFrame()

    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    
    # Merge with holiday data if available
    if not holiday.empty:
        df = df.merge(holiday[['date', 'Day_Type_2']], on='date', how='left')
        df.rename(columns={'Day_Type_2': 'Day_Type'}, inplace=True)
    else:
        df['Day_Type'] = 'regular'
    
    df['day_of_week'] = df['date'].dt.day_name()
    print(df.head(10))
    df.to_csv("TrainingData_Test.csv", index=False, encoding='utf-8-sig')
    model = joblib.load('best_model_XGBoost.pkl')
    encoder = joblib.load('onehot_encoder.pkl')
    
    # Try prediction if model is available
    try:
        X_new = df[['geohash5', 'geohash6', 'Day_Type','day_of_week', 'weather','hour','agegender']]
        X_new_encoded = encoder.transform(X_new)
        # Predict
        predictions = model.predict(X_new_encoded)
        
        # Store predictions
        df["predicted_impressions"] = predictions
        
    except Exception as e:
        print(f"Prediction error: {e}")
        df['predicted_impressions'] = np.random.randint(100, 1000, len(df))  # dummy data for testing

    # Merge with OTC ratio data
    df = df.merge(otcRatio, left_on="Reference_Id", right_on="referenceId", how="left")
    df["NonSpotImpressions"] = df["predicted_impressions"] * (df["share"] * df["mediaRatio"])
    df["NonSpotImpressions"] = df["NonSpotImpressions"].replace([np.inf, -np.inf], np.nan).fillna(0)

    # Calculate spot impressions
    est_spot_PV = []
    for _, row1 in df.iterrows():
        tmpSpot = custom_round(spot_calc(row1['NonSpotImpressions'], row1['dwellTime'], 
                                       row1['loopLength'], row1['spotsPerHour'], row1['spotDuration']))
        est_spot_PV.append(tmpSpot)

    df['SpotImpressions'] = est_spot_PV
    df['predicted_impressions'] = df['predicted_impressions'].round(2)
    df['NonSpotImpressions'] = df['NonSpotImpressions'].round(2)
    df['SpotImpressions'] = df['SpotImpressions']
    return df

def generate_report():
    global latest_report_data
    
    if not campaign_cart:
        return pd.DataFrame(columns=['Message']).assign(Message=['No items in cart. Please add items first.'])
    
    try:
        result_df = predict(campaign_cart)
        if result_df.empty:
            return pd.DataFrame(columns=['Message']).assign(Message=['No data generated. Please check your selections.'])
        
        # Store the full data for downloads
        latest_report_data = result_df.copy()
        
        # Return data in the specified format
        output_df = result_df[['Reference_Id', 'date', 'hour', 'agegender', 'predicted_impressions', 'NonSpotImpressions', 'SpotImpressions']].copy()
        
        # Rename columns to match the exact format requested
        output_df = output_df.rename(columns={
            'Reference_Id': 'ReferenceId',
            'predicted_impressions': 'predcited_impressions'  # keeping the typo as specified
        })
        
        # Sort by ReferenceId, date, hour, and agegender for better readability
        output_df = output_df.sort_values(['ReferenceId', 'date', 'hour', 'agegender']).reset_index(drop=True)
        
        return output_df
    except Exception as e:
        error_msg = f"Error generating report: {str(e)}"
        print(error_msg)
        return pd.DataFrame(columns=['Error']).assign(Error=[error_msg])

def download_full_data():
    """Download full data as CSV"""
    global latest_report_data

    if latest_report_data is None or latest_report_data.empty:
        return None

    try:
        # Prepare the data
        df = latest_report_data[['Reference_Id', 'date', 'hour', 'agegender', 'predicted_impressions', 'NonSpotImpressions', 'SpotImpressions']].copy()
        df = df.rename(columns={'Reference_Id': 'ReferenceId', 'predicted_impressions': 'predcited_impressions'})
        df['predcited_impressions'] = df['predcited_impressions'].round(2)
        df['NonSpotImpressions'] = df['NonSpotImpressions'].round(2)
        df['SpotImpressions'] = df['SpotImpressions'].round(2)

        # Create temporary file with proper extension
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.csv')
        df.to_csv(temp_file.name, index=False)
        temp_file.close()

        print(f"CSV generated: {temp_file.name}")
        return temp_file.name

    except Exception as e:
        print(f"Error saving CSV: {e}")
        return None


def download_excel_report():
    """Download the Excel report"""
    global latest_report_data
    
    if latest_report_data is None or latest_report_data.empty:
        return None
    
    try:
        # Prepare data for Excel report
        excel_df = latest_report_data.copy()
        excel_file = generate_excel_report(excel_df)
        
        if excel_file:
            print(f"Excel generated: {excel_file}")
            return excel_file
        else:
            return None
            
    except Exception as e:
        print(f"Error generating Excel: {e}")
        return None

# Create the Gradio interface
with gr.Blocks(title="Post Campaign Report Tool") as demo:
    gr.Markdown("# Post Campaign Report Data Generation Tool")
    
    with gr.Tab("Add Network Campaign"):
        with gr.Row():
            with gr.Column():
                name_n = gr.Dropdown(choices=network_names, label="Select Network Name", value=None)
                start_date_n = gr.Textbox(label="Start Date (YYYY-MM-DD)", placeholder="2024-01-01")
                end_date_n = gr.Textbox(label="End Date (YYYY-MM-DD)", placeholder="2024-01-31")
            with gr.Column():
                hour_type_n = gr.CheckboxGroup(
                    choices=['morning', 'afternoon', 'evening', 'night', 'full'], 
                    label="Hour Types",
                    value=[]
                )
                spots_n = gr.Number(label="Spots per Hour", value=1, minimum=1)
                btn_n = gr.Button("Add Network to Cart", variant="primary")
        
        out_n = gr.Textbox(label="Status Message")
    
    with gr.Tab("Add Location Campaign"):
        with gr.Row():
            with gr.Column():
                name_l = gr.Dropdown(choices=location_names, label="Select Location Name", value=None)
                start_date_l = gr.Textbox(label="Start Date (YYYY-MM-DD)", placeholder="2024-01-01")
                end_date_l = gr.Textbox(label="End Date (YYYY-MM-DD)", placeholder="2024-01-31")
            with gr.Column():
                hour_type_l = gr.CheckboxGroup(
                    choices=['morning', 'afternoon', 'evening', 'night', 'full'], 
                    label="Hour Types",
                    value=[]
                )
                spots_l = gr.Number(label="Spots per Hour", value=1, minimum=1)
                btn_l = gr.Button("Add Location to Cart", variant="primary")
        
        out_l = gr.Textbox(label="Status Message")
    
    with gr.Tab("Cart Management"):
        gr.Markdown("## Current Cart Items")
        cart_display = gr.Dataframe(label="Cart Contents", interactive=False)
        
        with gr.Row():
            with gr.Column(scale=3):
                remove_dropdown = gr.Dropdown(label="Select Item to Remove", choices=[], interactive=True)
            with gr.Column(scale=1):
                remove_btn = gr.Button("Remove Selected", variant="secondary")
            with gr.Column(scale=1):
                clear_btn = gr.Button("Clear All", variant="stop")
        
        cart_message = gr.Textbox(label="Cart Status", interactive=False)
    
    with gr.Tab("Generate Report"):
        gr.Markdown("## Generate Impression Report")
        gr.Markdown("Click the button below to generate a report based on all items in your cart.")
        
        with gr.Row():
            report_btn = gr.Button("Generate Report", variant="primary", size="lg")
        
        report_output = gr.Dataframe(label="Campaign Report", interactive=False)
        
        gr.Markdown("## Download Options")
        gr.Markdown("After generating a report, you can download the data in different formats:")
        
        with gr.Row():
            with gr.Column():
                gr.Markdown("### Full Data CSV")
                gr.Markdown("Download the complete dataset with all columns")
                download_csv_btn = gr.Button("Download Full Data (CSV)", variant="secondary")
                csv_download = gr.File(label="CSV Download", visible=False)
            
            with gr.Column():
                gr.Markdown("### Excel Report")
                gr.Markdown("Download comprehensive Excel report with multiple sheets and analysis")
                download_excel_btn = gr.Button("Download Excel Report", variant="secondary")
                excel_download = gr.File(label="Excel Download", visible=False)
    
    # Event handlers
    btn_n.click(
        fn=add_to_cart,
        inputs=[gr.Textbox(value="Network", visible=False), name_n, start_date_n, end_date_n, hour_type_n, spots_n],
        outputs=[out_n, cart_display, remove_dropdown]
    )
    
    btn_l.click(
        fn=add_to_cart,
        inputs=[gr.Textbox(value="Location", visible=False), name_l, start_date_l, end_date_l, hour_type_l, spots_l],
        outputs=[out_l, cart_display, remove_dropdown]
    )
    
    remove_btn.click(
        fn=remove_from_cart,
        inputs=[remove_dropdown],
        outputs=[cart_message, cart_display, remove_dropdown]
    )
    
    clear_btn.click(
        fn=clear_cart,
        outputs=[cart_message, cart_display, remove_dropdown]
    )
    
    # Updated event handlers - replace the existing download event handlers
    report_btn.click(
        fn=generate_report,
        outputs=[report_output]
    )
    
    # CSV Download
    download_csv_btn.click(
        fn=download_full_data,
        outputs=[csv_download]
    ).then(
        fn=lambda x: gr.update(visible=True) if x else gr.update(visible=False),
        inputs=[csv_download],
        outputs=[csv_download]
    )
    
    # Excel Download  
    download_excel_btn.click(
        fn=download_excel_report,
        outputs=[excel_download]
    ).then(
        fn=lambda x: gr.update(visible=True) if x else gr.update(visible=False),
        inputs=[excel_download],
        outputs=[excel_download]
    )
    
    # Load initial cart display
    demo.load(
        fn=lambda: (get_cart_display(), update_remove_choices()),
        outputs=[cart_display, remove_dropdown]
    )

if __name__ == '__main__':
    demo.launch(debug=True, share=True)

All files loaded successfully!
* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


        date           Reference_Id geohash5 geohash6  hour       agegender  \
0 2025-06-01  JPN-JEK-D-00000-00029    xn777   xn777c     5    female_10_19   
1 2025-06-01  JPN-JEK-D-00000-00029    xn777   xn777c     5    female_20_29   
2 2025-06-01  JPN-JEK-D-00000-00029    xn777   xn777c     5    female_30_39   
3 2025-06-01  JPN-JEK-D-00000-00029    xn777   xn777c     5    female_40_49   
4 2025-06-01  JPN-JEK-D-00000-00029    xn777   xn777c     5    female_50_59   
5 2025-06-01  JPN-JEK-D-00000-00029    xn777   xn777c     5  female_60_plus   
6 2025-06-01  JPN-JEK-D-00000-00029    xn777   xn777c     5      male_10_19   
7 2025-06-01  JPN-JEK-D-00000-00029    xn777   xn777c     5      male_20_29   
8 2025-06-01  JPN-JEK-D-00000-00029    xn777   xn777c     5      male_30_39   
9 2025-06-01  JPN-JEK-D-00000-00029    xn777   xn777c     5      male_40_49   

  weather  spotsPerHour  spotDuration  dwellTime  loopLength  \
0    rain            10            15         10         360   
1 

In [28]:
print(type(encoder))  # Should be <class 'sklearn.preprocessing._encoders.OneHotEncoder'>

<class 'numpy.ndarray'>


In [34]:
location_df

,ReferenceId,spotsPerHour,dwellTime,loopLength,spotDuration,lat,lon,geohash5,geohash6
0,JPN-JEK-D-00000-00029,10,10,360,15,35.733539,139.739261,xn777,xn777c
1,JPN-JEK-D-00000-00030,10,10,360,15,35.691599,139.701027,xn774,xn774c
2,JPN-JEK-D-00000-00031,10,10,360,15,35.689607,139.700571,xn774,xn774c
3,JPN-JEK-D-00000-00032,10,10,360,15,35.688715,139.700862,xn774,xn774b
4,JPN-JEK-D-00000-00034,10,10,360,15,35.626069,139.723606,xn76e,xn76em
5,JPN-JEK-D-00000-00035,10,10,360,15,35.628471,139.738760,xn76e,xn76ev
6,JPN-JEK-D-00000-00039,10,10,360,15,35.674765,139.763051,xn76u,xn76uq
7,JPN-JEK-D-00000-00040,10,10,360,15,35.681846,139.765255,xn76u,xn76ur
8,JPN-JEK-D-00000-00041,10,10,360,15,35.679865,139.767138,xn76u,xn76ur
9,JPN-JEK-D-00000-00042,10,10,360,15,35.698444,139.772554,xn77h,xn77hd


In [36]:
get_weather_label('2025-05-01', 35.655697,139.338959)

'sunny'